<a href="https://colab.research.google.com/github/Shrinjita/GenAI-LLM/blob/ChatGLM-Implemention/ChatGLM_Implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#INSTALL REQUIRED LIBRARIES
!pip install transformers==4.27.1
!pip install sentencepiece
!pip install cpm_kernels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 6.6 MB/s eta 0:00:00


In [ ]:
#CHECK GPU INFO
!nvidia-smi

Sat Oct 14 04:48:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b-int4", trust_remote_code=True)

(…)-int4/resolve/main/tokenizer_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


(…)nt4/resolve/main/tokenization_chatglm.py:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

In [ ]:
model = AutoModel.from_pretrained("THUDM/chatglm2-6b-int4", trust_remote_code=True).half().cuda()

(…)hatglm2-6b-int4/resolve/main/config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


(…)t4/resolve/main/configuration_chatglm.py:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


(…)6b-int4/resolve/main/modeling_chatglm.py:   0%|          | 0.00/54.9k [00:00<?, ?B/s]

(…)lm2-6b-int4/resolve/main/quantization.py:   0%|          | 0.00/2.61M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.92G [00:00<?, ?B/s]

In [ ]:
data = """

From conversation below, list subjective items. Under each subjective item, list all properties (both label and value) as child bullet points such as (in this specific order)
Subjective (generate in inference style sentence, in as few words as possible), and include severity and body part if part of the conversation.
Interpretation (whether the finding is normal, abnormal, inconclusive, or uncertain)
Count (translate to numbers, only if mentioned in conversation)
Laterality
Body Part (if found in conversation)
Inferenced anatomical part
Characterization
Severity
Onset (when did it start)
Frequency
Probable Cause
Aggravating Factors
Alleviating Factors
Duration (including temporal factors
Conversational Quote that inspired this text
Keywords or Key Phrases from the conversation that are most relevant
Summary (description of this subjective item in a sentence or two, in paragraph format, in as few words as possible, but do not include veterinarian response or recommendations here).

For subjective findings, do not only consider with question asked, but also check for a response and then summarize the subjective finding in the appropriate fashion.

Publish as a JSON array as per the example below:
{
"Subjective Item 1": {
"Subjective": "",
"Interpretation": "",
"Laterality": "",
"Characterization": "",
"Severity": "",
"Onset": "",
"Frequency": "",
"Probable Cause": "",
"Aggravating Factors": "",
"Alleviating Factors": ""
}


Doctor: Good morning! How can I assist you today?

Patient: Hi, I'm not feeling well. I have been having a persistent headache for the past two days.

Doctor: I see. Can you tell me a little more about your headache? Where is the pain located, and does it feel like a sharp or dull ache?

Patient: It's a dull ache that's mainly located in the back of my head. It's been pretty constant for the past two days.

Doctor: Alright. Based on your symptoms, I would like to prescribe you some medication. I recommend taking paracetamol 500mg, twice a day for three days. This medication is commonly used to treat headaches and can help to relieve your symptoms. Do you have any allergies to medications?

Patient: No, I don't have any allergies.

Doctor: Great. In addition to the paracetamol, I would also recommend that you drink plenty of water and get some rest. Dehydration can sometimes cause headaches, so it's important to stay hydrated. Additionally, getting enough rest and avoiding stress can also help to alleviate your symptoms.

Patient: Alright, that sounds like a good plan. Is there anything else I should know about the medication?

Doctor: Yes, it's important that you take the medication exactly as prescribed. Do not take more than the recommended dose, as this can be dangerous. If you have any questions or concerns about the medication, please don't hesitate to ask.

Patient: Okay, I'll be sure to follow the instructions. How long should I take the medication for?

Doctor: You should take the medication for three days, as I mentioned earlier. If your headache persists after three days or if you experience any other symptoms, please come back and see me.

Patient: Alright, thank you, doctor.

Doctor: You're welcome. Please take the prescription to the pharmacy and they will be able to fill it for you. Take care and feel better soon.

"""

In [ ]:
response, history = model.chat(tokenizer,data , history=[])

In [ ]:
print(response)

{
"Subjective Item 1": {
"Subjective": "Patient not experiencing persistent headache",
"Interpretation": "Patient is experiencing a persistent headache that is located in the back of the head and feels like a dull ache",
"Laterality": "",
"Characterization": "Patient is experiencing a persistent headache that is located in the back of the head and feels like a dull ache",
"Severity": "Patient is experiencing a persistent headache that is located in the back of the head and feels like a dull ache",
"Onset": "Patient is experiencing a persistent headache that is located in the back of the head and feels like a dull ache for the past two days",
"Frequency": "Patient is experiencing a persistent headache that is located in the back of the head and feels like a dull ache twice a day for three days",
"Probable Cause": "",
"Aggravating Factors": "",
"Alleviating Factors": ""
}


In [ ]:
data = """
Speaker 0: Good morning! How can I assist you today?

Speaker 1: Hi, I'm not feeling well. I have been having a persistent headache for the past two days.

Speaker 0: I see. Can you tell me a little more about your headache? Where is the pain located, and does it feel like a sharp or dull ache?

Speaker 1: It's a dull ache that's mainly located in the back of my head. It's been pretty constant for the past two days.

Speaker 0: Alright. Based on your symptoms, I would like to prescribe you some medication. I recommend taking paracetamol 500mg, twice a day for three days. This medication is commonly used to treat headaches and can help to relieve your symptoms. Do you have any allergies to medications?

Speaker 1: No, I don't have any allergies.

Speaker 0: Great. In addition to the paracetamol, I would also recommend that you drink plenty of water and get some rest. Dehydration can sometimes cause headaches, so it's important to stay hydrated. Additionally, getting enough rest and avoiding stress can also help to alleviate your symptoms.

Speaker 1: Alright, that sounds like a good plan. Is there anything else I should know about the medication?

Speaker 0: Yes, it's important that you take the medication exactly as prescribed. Do not take more than the recommended dose, as this can be dangerous. If you have any questions or concerns about the medication, please don't hesitate to ask.

Speaker 1: Okay, I'll be sure to follow the instructions. How long should I take the medication for?

Speaker 0: You should take the medication for three days, as I mentioned earlier. If your headache persists after three days or if you experience any other symptoms, please come back and see me.

Speaker 1: Alright, thank you, doctor.

Speaker 0: You're welcome. Please take the prescription to the pharmacy and they will be able to fill it for you. Take care and feel better soon.

Were any vaccine names and statuses discussed in this conversation?
"""

In [ ]:
data = """
Write a story about grey morals.
"""

In [ ]:
response, history = model.chat(tokenizer,data , history=[])

In [ ]:
print(response)

Once upon a time in a land far, far away, there lived a wise and powerful grey moral. This grey moral was known as \"Grey\". Grey was a being of great moral weight, and was revered by all who knew him.

Grey was a shining light in a world that was often shrouded in darkness. He was a beacon of hope, and a reminder of the beauty of the world. People would come to him for guidance, and for the strength of his moral.

One day, a young girl came to Grey. She was filled with wonder and fear, and she had been told that she was to be married the following day. Grey took the young girl under his wing, and he showed her the beauty of the world, and the strength of his moral.

\"Fear not, my dear child,\" Grey said to her. \"For I have seen the darkness that you have been told of, and I have come to tell you that there is also the light. The light of reason, and the light of love. You have a right to choose your own path, and to follow your own heart. Do not be afraid, for you are strong, and yo

In [ ]:
data = """
Speaker 0: Good morning! How can I assist you today?

Speaker 1: Hi, I'm not feeling well. I have been having a persistent diarrhea for the past two days.

Speaker 0: I see. Can you tell me a little more about your headache? Where is the pain located, and does it feel like a sharp or dull ache?

Speaker 1: It's a dull ache that's mainly located in the back of my head. It's been pretty constant for the past two days.

Speaker 0: Alright. Based on your symptoms, I would like to prescribe you some medication. I recommend taking Imodium 400 g, three times a day for 7 days. This medication is commonly used to treat diarrhea and can help to relieve your symptoms. Do you have any allergies to medications?

Speaker 1: No, I don't have any allergies.

Speaker 0: Great. In addition to the paracetamol, I would also recommend that you drink plenty of water and get some rest. Dehydration can sometimes cause headaches, so it's important to stay hydrated. Additionally, getting enough rest and avoiding stress can also help to alleviate your symptoms.

Speaker 1: Alright, that sounds like a good plan. Is there anything else I should know about the medication?

Speaker 0: Yes, it's important that you take the medication exactly as prescribed. Do not take more than the recommended dose, as this can be dangerous. If you have any questions or concerns about the medication, please don't hesitate to ask.

Speaker 1: Okay, I'll be sure to follow the instructions. How long should I take the medication for?

Speaker 0: You should take the medication for three days, as I mentioned earlier. If your headache persists after three days or if you experience any other symptoms, please come back and see me.

Speaker 1: Alright, thank you, doctor.

Speaker 0: You're welcome. Please take the prescription to the pharmacy and they will be able to fill it for you. Take care and feel better soon.

List the drugs and their dosages if any has been mentioned in the above conversation in the following JSON format:
{
 "drugs":"",
 "Dosage":"",
 "Frequency":"",
 "Duration":""
}
"""

In [ ]:
response, history = model.chat(tokenizer,data , history=[])

In [ ]:
print(response)

{
"drugs": "Imodium 400 g, three times a day for 7 days",
"Dosage": "400 mg",
"Frequency": "three times a day",
"Duration": "7 days"
}


In [ ]:
data = """
I want you to write a program in java to check for palindromic numbers.
"""

In [ ]:
response, history = model.chat(tokenizer,data , history=[])

In [ ]:
print(response)

Sure, here is a Java program that checks for palindromic numbers:

``` 
import java.util.Scanner; 

public class PalindromeChecker {
    public static void main(String[] args) {
        Scanner input = new Scanner(System.in);
        System.out.print("Enter a number: ");
        int number = input.nextInt();

        boolean isPalindrome = true;
        for (int i = 0; i < number.length(); i++) {
            char c = number.charAt(i);
            if (c == '0' || c == '1' || c == '2' || c == '3' || c == '4' || c == '5' ||
                c == '6' || c == '7' || c == '8' || c == '9') {
                isPalindrome = false;
                break;
            }
        }

        System.out.println("Is this a palindrome? " + isPalindrome);
    }
}
```

This program first ask the user to enter a number. It then uses a for loop to check the characters in the number. If the number is a palindrome, it will have the same number of even and odd characters. If the program finds that the number is